In [6]:
import time
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [7]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5605672&myY=126.9433486&searchKeyword='
driver=webdriver.Chrome()
driver.get(url)

In [8]:
driver.find_element(By.ID,'searchKeyword').send_keys('서울') #'searchKeyword'라는 ID를 가진 요소에 '서울'이라는 텍스트 입력
driver.find_element(By.ID,'fsSearchBtn').click() #'sfsSearchBtn'라는 ID를 가진 요소를 클릭

In [9]:
soup=BeautifulSoup(driver.page_source,'html.parser') #셀레늄을 사용해 웹페이지의 HTML소스코드를 BeautifulSoup으로 파싱
lis=soup.select('.stations-list>li')
len(lis)

10

In [10]:
li=lis[0]
li

<li id="0" style="height: 172px;">
<img class="marker" src="/resources/img/firestation/map_ic_station.png"/>
<span class="title">동작소방서</span>
<address>서울특별시 동작구 여의대방로16길 55(신대방동)</address>
<span class="tel">02-847-1190</span>
<span class="info">서울소방재난본부</span>
</li>

In [12]:
title=li.select_one('.title').get_text().strip() #선택된 요소에서 텍스트를 가져와 공백을 제거
title

'동작소방서'

In [13]:
address=li.select_one('address').get_text().strip()
address

'서울특별시 동작구 여의대방로16길 55(신대방동)'

In [14]:
tel=li.select_one('.tel').get_text().strip()
tel

'02-847-1190'

In [15]:
data=[]
for li in lis:
    title=li.select_one('.title').get_text().strip()
    address=li.select_one('address').get_text().strip()
    tel=li.select_one('.tel').get_text().strip()
    data.append({'기관명':title, '주소':address, '전화번호':tel})

In [16]:
data

[{'기관명': '동작소방서', '주소': '서울특별시 동작구 여의대방로16길 55(신대방동)', '전화번호': '02-847-1190'},
 {'기관명': '서대문소방서', '주소': '서울특별시 서대문구 연희로 182(연희동)', '전화번호': '02-3144-1190'},
 {'기관명': '광진소방서', '주소': '서울특별시 광진구 광나루로 480(구의동)', '전화번호': '02-457-0119'},
 {'기관명': '송파소방서', '주소': '서울특별시 송파구 오금로51길 56(마천동)', '전화번호': '02-403-2119'},
 {'기관명': '양천소방서', '주소': '서울특별시 양천구 목동서로 180(목동)', '전화번호': '02-2655-1119'},
 {'기관명': '은평소방서', '주소': '서울특별시 은평구 통일로 962(진관동)', '전화번호': '02-355-0119'},
 {'기관명': '노원소방서', '주소': '서울특별시 노원구 한글비석로 1길 8(하계동)', '전화번호': '02-977-0119'},
 {'기관명': '종로소방서', '주소': '서울특별시 종로구 종로1길 28(수송동)', '전화번호': '02-735-6119'},
 {'기관명': '도봉소방서', '주소': '서울특별시 도봉구 도봉로 666(방학동)', '전화번호': '02-3492-3437'},
 {'기관명': '용산소방서', '주소': '서울특별시 용산구 한강대로 167(한강로2가)', '전화번호': '02-794-0119'}]

In [17]:
driver.find_element(By.CSS_SELECTOR, '.next_page').click()

모든 페이지에 적용

In [18]:
#첫번째 페이지
driver.get(url)
time.sleep(2)
driver.find_element(By.ID,'searchKeyword').send_keys('서울')
driver.find_element(By.ID,'fsSearchBtn').click()

In [20]:
#서울 100건
count = driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/div/p/strong[2]').text
count = int(count[:-1]) #'건'뻬고
count

#'"서울" 소방기관 검색결과 100건' 여기서 '100건' XPATH
#특정 XPATH를 통해 요소를 찾고- > 해당 요소의 텍스트를 가져와서 문자열에서 마지막 문자를 제외한 부분을 정수형으로 변환 -> count변수에 저장

100

In [22]:
import math #총 페이지 수를 계산
total_pages=math.ceil(count/10) #math.ceil(): 소수점 이하를 올림하여 필요한 전체 페이지 수를 계산
total_pages

10

In [24]:
data1 = []
for i in range(total_pages): #변수에 저장된 총 페이지 수만큼 반복문 실행
    if i % 2 == 1: #만약 i가 홀수일 때
        driver.find_element(By.CSS_SELECTOR, '.next_page').click()
        time.sleep(1)
    if i > 0 and i % 2 == 0: #만약 i가 0보다 크고 짝수일 때
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[5]/a').click()
        time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list > li')
    for li in lis:
        title = li.select_one('.title').get_text().strip()
        addr = li.select_one('address').get_text().strip()
        tel = li.select_one('.tel').get_text().strip()
        data1.append({'기관명':title, '주소' :addr, '전화번호' : tel})

In [25]:
df1=pd.DataFrame(data1)
df1.head()

,기관명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119


In [26]:
len(df1)

100

In [27]:
df1.to_csv('서울소방서.csv', index=False) #인덱스를 CSV파일에 포함하지 않도록 지정